# 概要
ここでは、GoogleColaboratoryのGPUとGoogleDriveを使用して、BERTのチュートリアルを動かします。  
BERTのチュートリアルをでは、`MRPC`という文章の等価性を評価するタスクを解いていますので、ここでもこれを解いていきます。  
  
GoogleDriveに学習済みモデルがデータセットを保存するため、`500MB`の空きが必要です。

## 全体の流れ

1. Colaboratory notebookの設定
1. GoogleDriveをマウント
1. データセット/BERT学習済みモデルをGoogleDriveへ保存
1. BERTのリポジトリをClone
1. trainデータを使用した、学習済みモデルのfine-tuning
1. testデータを使用した、データ予測
1. 予測結果(出力)の確認

### 1.Colaboratory notebookの設定
当notebookをGPUで実行できるように設定を行います。  
Qiitaの記事を参考にすることをおすすめします。  
1. 上の`ランタイム` -> `ランタイムのタイプを変更`を押す
1. `ノートブックの設定`にて、`ハードウェアのアクセラレータ`を`GPU`に変更
1. 右上の`接続`を押し、`RAM/ディスク`になるまで待つ

### 2.GoogleDriveをマウント

In [2]:
from google.colab import drive 


drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [45]:
# 作業ディレクトリをDrive上に作成
!mkdir drive/'My Drive'/bert

mkdir: cannot create directory ‘bert’: File exists


In [3]:
# 作業ディレクトリへ移動
cd drive/'My Drive'/bert

/content/drive/My Drive


### 3.データセット/BERT学習済みモデルをGoogleDriveへ保存

#### データセットの保存
GLUEデータセットのMRPCタスクのみ保存を行います。

In [48]:
# DLスクリプトの取得
!git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git

Cloning into '60c2bdb54d156a41194446737ce03e2e'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), done.


In [50]:
!python 60c2bdb54d156a41194446737ce03e2e/download_glue_data.py \
--data_dir glue_data --tasks MRPC

Processing MRPC...
Local MRPC data not specified, downloading data from https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt
	Completed!


#### BERT学習済みモデルの保存
公式が公開している`BERT-Base, Uncasedモデル`を保存します。

In [54]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip && \
unzip uncased_L-12_H-768_A-12.zip && \
rm uncased_L-12_H-768_A-12.zip

--2020-01-05 09:03:47--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 2404:6800:4003:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M  95.2MB/s    in 4.0s    

2020-01-05 09:03:51 (96.2 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


### BERTのリポジトリをClone
BERTリポジトリ内の`run_classifier.py`を実行してタスクを解くため、リポジトリをCloneします。

In [5]:
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 287.79 KiB | 887.00 KiB/s, done.
Resolving deltas: 100% (184/184), done.


## trainデータを使用した、学習済みモデルのfine-tuning
glue_data/MRPC/train.tsvを使用し、学習済みモデルのfine-tuningを行います。

##### パラメーターの説明
- task_name: 解くタスクを指定
 - MRPC: 2つのニュース記事の等価性を判定
 - CoLa: 文章が文法的に正しいか判定
 - MNLI: テキスト同士の関連性を判定
 - XNLI: 翻訳系のタスク(?)
- do_train: 学習するか否か
- do_eval: 評価結果を表示するか否か
- data_dir: データセット(MRPC)のディレクトリPath
- vocab_file: 学習済みモデルのvocab.txtのPath
- bert_config_file: 学習済みモデルのbert_config.jsonのPath
- init_checkpoint: 学習済みモデルのbert_model.ckpt.***のPath
- max_seq_length: 最長トークンの長さ
- train_batch_size: 学習Batchサイズ
- learning_rate: 学習率
- num_train_epochs: エポック数
- output_dir: 結果出力先

In [6]:
!python bert/run_classifier.py \
  --task_name=MRPC \
  --do_train=true \
  --do_eval=true \
  --data_dir=glue_data/MRPC \
  --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=2e-5 \
  --num_train_epochs=3.0 \
  --output_dir=tmp/mrpc_output/




W0105 10:33:57.009868 140299170789248 module_wrapper.py:139] From bert/run_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0105 10:33:57.010150 140299170789248 module_wrapper.py:139] From bert/run_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0105 10:33:57.010623 140299170789248 module_wrapper.py:139] From /content/drive/My Drive/bert/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0105 10:33:57.750652 140299170789248 module_wrapper.py:139] From bert/run_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https

完了すると以下の出力がされます。

```
INFO:tensorflow:***** Eval results *****
I0105 10:34:39.428679 140299170789248 run_classifier.py:923] ***** Eval results *****
INFO:tensorflow:  eval_accuracy = 0.85294116
I0105 10:34:39.428822 140299170789248 run_classifier.py:925]   eval_accuracy = 0.85294116
INFO:tensorflow:  eval_loss = 0.4425803
I0105 10:34:40.041571 140299170789248 run_classifier.py:925]   eval_loss = 0.4425803
INFO:tensorflow:  global_step = 343
I0105 10:34:40.041978 140299170789248 run_classifier.py:925]   global_step = 343
INFO:tensorflow:  loss = 0.4425803
I0105 10:34:40.042183 140299170789248 run_classifier.py:925]   loss = 0.4425803
```

### testデータを使用した、データ予測
test.tsvを使用して、予測を行います。

##### パラメーターの説明
- do_predict: 予測を行うか否か
- init_checkpoint: fine-tuning後のbert_model.ckpt.***のPath
- output_dir: 予測結果の出力先

In [7]:
!python bert/run_classifier.py \
  --task_name=MRPC \
  --do_predict=true \
  --data_dir=glue_data/MRPC \
  --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=tmp/mrpc_output \
  --max_seq_length=128 \
  --output_dir=tmp/mrpc_output_result/




W0105 10:42:00.557227 140335141369728 module_wrapper.py:139] From bert/run_classifier.py:784: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0105 10:42:00.557511 140335141369728 module_wrapper.py:139] From bert/run_classifier.py:784: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0105 10:42:00.558079 140335141369728 module_wrapper.py:139] From /content/drive/My Drive/bert/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0105 10:42:00.559884 140335141369728 module_wrapper.py:139] From bert/run_classifier.py:808: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https

### 予測結果(出力)の確認
GoogleDriveの`bert/tmp/mrpc_output_result/`に、`test_results.tsv`が出力されています。

以上